In [1]:
!pip install "jax[cuda12_local]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install transformers scipy ftfy accelerate
!pip install diffusers==0.11.1
!pip install flask pyngrok

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [6]:
# Forse non serve, provare a runnare il resto senza runnare questa cella
!wget -q -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip
!ngrok config add-authtoken 2hCAtAEtjMYj0RzyLs4V8lN3QE7_6fg61ry6eQuCWnpeynvSL

Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from flask import Flask
from pyngrok import ngrok
import time

2024-05-30 17:12:11.270483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 17:12:11.270547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 17:12:11.272045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.to("cuda")

In [ ]:
prompt = "an image of the sea with sunrise"

image = pipe(prompt).images[0]

In [ ]:
display(image)

In [ ]:
image.save("/kaggle/working/image.png")

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_cols = 2
num_rows = 1

prompt = ["a photograph of an astronaut riding a horse"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

In [5]:
!pip install flask pyngrok

In [10]:
from flask import Flask
from pyngrok import ngrok

In [11]:
app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, World!"

@app.route('/image', methods=['POST'])
def get_image():
    try:
        data = request.get_json()
        #prompt = data.get('prompt', '')
        prompt = data['prompt']
        print(prompt)
        image = pipe(prompt).images[0]
        #time.sleep(200000)
        
        img_io = io.BytesIO()
        image.save(img_io, 'JPEG')
        img_io.seek(0)
        
        return send_file(img_io, mimetype='image/jpeg')
        
    except Exception as e:
        app.logger.error(f"Errore durante la generazione o l'invio dell'immagine: {e}")
        #abort(500)

@app.route('/mao', methods=['POST'])
def miao():
    data = request.json
    prompt = data['prompt']
    print(prompt)
    image = pipe(prompt).images[0]
    try:
        image.save("/kaggle/working/mirimao.png")
        return send_file("/kaggle/working/mirimao.png", as_attachment=True, download_name='image.png')
    except Exception as e:
        return str(e)
    
@app.route('/porco', methods=['POST'])
def porco():
    return send_file("/kaggle/working/image.png", as_attachment=True, download_name='image.png')


if __name__ == '__main__':
    port = 5000

    # Avvia ngrok
    ngrok.set_auth_token("2hCAtAEtjMYj0RzyLs4V8lN3QE7_6fg61ry6eQuCWnpeynvSL")  # Configura il tuo authtoken
    public_url = ngrok.connect(port).public_url
    print(f"Server pubblico URL: {public_url}")

    # Avvia Flask
    app.run(port=port, debug=True)


Server pubblico URL: https://0960-34-91-94-246.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 654, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/opt/conda/lib/python3.10/site-packages/

SystemExit: 1